# 1. Labeling images for Object Detection

* labelimg 패키지 이용하여 이미지 라벨링하기

In [ ]:
# 작업 폴더 설정 후 anaconda prompt에서 작업


# 샘플 프로젝트 리포지토리와  복제
git clone https://github.com/nicknochnack/RealTimeObjectDetection

# 이미지 라벨을 지정하는 파이썬 패키지, 복제하여 RealTimeObjectDetection>Tensorflow 에 넣기
git clone https://github.com/tzutalin/labelImg

# labelimg 폴더로 이동 후 몇가지 종속성 설치 (https://wonderbout.tistory.com/185)

pip install pyqt5-tools 윈도우 아나콘다 유저

"""----------------------------------------------------------------------------------------"""

#작업 폴더 설정 후 anaconda prompt에서 작업

#< 샘플 프로젝트 clone>

# 1.샘플 프로젝트 리포지토리와  복제
git clone https://github.com/nicknochnack/RealTimeObjectDetection

"""----------------------------------------------------------------------------------------"""
#<imagelabel 설치>
#이미지 라벨을 지정하는 파이썬 패키지, 복제하여 RealTimeObjectDetection>Tensorflow 에 넣기 
git clone https://github.com/tzutalin/labelImg

l#abelimg 폴더로 이동 후 두가지 종속성 설치 (https://wonderbout.tistory.com/185)

#(1) pip install pyqt5-tools (윈도우 아나콘다 유저 경우) 혹은 conda install pyqt=5
#(2) conda -c anaconda lxml resource.qrc 파일을 .py로 복제하여 라이브러리 폴더에 넣기(두개 다) prcc5 -o resources.py resources.qrc

"""----------------------------------------------------------------------------------------"""
#< 이미지 다운로드 받아서 workspace > images에 넣기>
#https://www.kaggle.com/datasets/wobotintelligence/face-mask-detection-dataset

#프롬프트에서 python labelImg.py 로 패키지 실행
#(1) view - 자동 저장 켜주기
#(2) 저장 폴더를 아까 이미지 모아둔 폴더로 바꿔주기
"""----------------------------------------------------------------------------------------"""
#python labelImg
#(1) view에서 자동 저장 확인
#(2) change save Dir 하여 아까 이미지 모아둔 폴더로 선택
#(3) open Dir해서 사진 있는 폴더 선택

#w : 라벨링 지정 툴
#a / d : 이미지 넘기기


#이 과정에서 labelImg 창이 닫힌다면 공식 labelImg git 페이지에 가서 설치 다시 해볼 것
#(해결 : pip3 install labelImg
# python labelImg.py [IMAGE_PATH] [PRE-DEFINED CLASS FILE])

#(4) w 눌러 사진들 입주변 드래그 해서 라벨링해준다  - 라벨링된 파일들이 경로에 생성
#(5) 이제 교육과 테스트 용도로 분류하여 파일에 나눠 넣어준다 ( 원본 이미지와 함께) 



# 2. Training Tensorflow SSD
### tensorflow 객체 감지 API를 사용하여 딥 러닝 모델 훈련하기 
* SSD MobileNet을 이용하여 전이 학습(transfer learning)

In [ ]:
#  -----------------------------generate_tfrecord를 tensor flow v2에 맞게 수정-------------------

""" Sample TensorFlow XML-to-TFRecord converter

usage: generate_tfrecord.py [-h] [-x XML_DIR] [-l LABELS_PATH] [-o OUTPUT_PATH] [-i IMAGE_DIR] [-c CSV_PATH]

optional arguments:
  -h, --help            show this help message and exit
  -x XML_DIR, --xml_dir XML_DIR
                        Path to the folder where the input .xml files are stored.
  -l LABELS_PATH, --labels_path LABELS_PATH
                        Path to the labels (.pbtxt) file.
  -o OUTPUT_PATH, --output_path OUTPUT_PATH
                        Path of output TFRecord (.record) file.
  -i IMAGE_DIR, --image_dir IMAGE_DIR
                        Path to the folder where the input image files are stored. Defaults to the same directory as XML_DIR.
  -c CSV_PATH, --csv_path CSV_PATH
                        Path of output .csv file. If none provided, then no file will be written.
"""

import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple

# Initiate argument parser
parser = argparse.ArgumentParser(
    description="Sample TensorFlow XML-to-TFRecord converter")
parser.add_argument("-x",
                    "--xml_dir",
                    help="Path to the folder where the input .xml files are stored.",
                    type=str)
parser.add_argument("-l",
                    "--labels_path",
                    help="Path to the labels (.pbtxt) file.", type=str)
parser.add_argument("-o",
                    "--output_path",
                    help="Path of output TFRecord (.record) file.", type=str)
parser.add_argument("-i",
                    "--image_dir",
                    help="Path to the folder where the input image files are stored. "
                         "Defaults to the same directory as XML_DIR.",
                    type=str, default=None)
parser.add_argument("-c",
                    "--csv_path",
                    help="Path of output .csv file. If none provided, then no file will be "
                         "written.",
                    type=str, default=None)

args = parser.parse_args()

if args.image_dir is None:
    args.image_dir = args.xml_dir

label_map = label_map_util.load_labelmap(args.labels_path)
label_map_dict = label_map_util.get_label_map_dict(label_map)


def xml_to_csv(path):
    """Iterates through all .xml files (generated by labelImg) in a given directory and combines
    them in a single Pandas dataframe.

    Parameters:
    ----------
    path : str
        The path containing the .xml files
    Returns
    -------
    Pandas DataFrame
        The produced dataframe
    """

    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def class_text_to_int(row_label):
    return label_map_dict[row_label]


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    writer = tf.io.TFRecordWriter(args.output_path)
    path = os.path.join(args.image_dir)
    examples = xml_to_csv(args.xml_dir)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('Successfully created the TFRecord file: {}'.format(args.output_path))
    if args.csv_path is not None:
        examples.to_csv(args.csv_path, index=None)
        print('Successfully created the CSV file: {}'.format(args.csv_path))


if __name__ == '__main__':
    tf.compat.v1.app.run()

# ---------------------------- 오류 수정 실패 -----------------------------------

# 3. Real Time Detection
* Jupyter notebook을 이용하여 openCV 및 Python을 활용한 실시간 감지